In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('/content/drive/MyDrive/datasets/subtask_2/es/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text label
0  12786  Sin embargo, los jóvenes son capaces de recono...     B
1  12361  ¿Hay algo más que quieras compartir? ¿Algo sob...     B
2   1662  El servicio de sala es bueno, rápido y amabilí...     B
3  14729  Para concentrarse en el hablante, trata de des...     F
4   9312  Los responsables locales tendrán ahora que esp...     F


In [5]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
print("Unique labels: ", set(train_data_labels))


Unique labels:  {'C', 'F', 'A', 'B', 'E', 'D'}


In [6]:
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))

train data size:  17766
validation data size:  1975
test data size:  2194


In [7]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [8]:
import nltk
# feature augmentation using pos tags
nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
tagger = nltk.pos_tag

train_texts_pos = []
for text in train_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    train_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    train_texts_pos.append(train_pos_tags)

val_texts_pos = []
for text in val_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    val_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    val_texts_pos.append(val_pos_tags)

test_texts_pos = []
for text in test_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    test_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    test_texts_pos.append(test_pos_tags)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
from textblob import TextBlob

# Create a list of sentiment scores for each text
train_sentiments = [TextBlob(text).sentiment.polarity for text in train_texts]
val_sentiments = [TextBlob(text).sentiment.polarity for text in val_texts]
test_sentiments = [TextBlob(text).sentiment.polarity for text in test_texts]

# Convert the sentiment scores to a tensor
train_sentiments = torch.tensor(train_sentiments).unsqueeze(-1)
val_sentiments = torch.tensor(val_sentiments).unsqueeze(-1)
test_sentiments = torch.tensor(test_sentiments).unsqueeze(-1)

# Convert to a list of strings
train_texts_pos = [' '.join(pos_tags) for pos_tags in train_texts_pos]
val_texts_pos = [' '.join(pos_tags) for pos_tags in val_texts_pos]
test_texts_pos = [' '.join(pos_tags) for pos_tags in test_texts_pos]

train_encodings = tokenizer(train_texts, train_texts_pos, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, val_texts_pos, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, test_texts_pos, truncation=True, padding=True, max_length=256)

# Concatenate the sentiment scores with the BERT embeddings
train_encodings.update({'sentiment': train_sentiments})
val_encodings.update({'sentiment': val_sentiments})
test_encodings.update({'sentiment': test_sentiments})

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels[idx] == 'A':
            item['labels'] = torch.tensor(0)
        elif self.labels[idx] == 'B':
            item['labels'] = torch.tensor(1)
        elif self.labels[idx] == 'C':
            item['labels'] = torch.tensor(2)
        elif self.labels[idx] == 'D':
            item['labels'] = torch.tensor(3)
        elif self.labels[idx] == 'E':
            item['labels'] = torch.tensor(4)
        elif self.labels[idx] == 'F':
            item['labels'] = torch.tensor(5)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

print("Sample Data Point")
print()
print(train_dataset[0])

'''
  to rectify the warning:
  <ipython-input-9-a0a6613eed66>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).detach().clone() for key, val in self.encodings.items()}
        if self.labels[idx] == 'human':
            item['labels'] = torch.tensor(0).detach().clone()
        else:
            item['labels'] = torch.tensor(1).detach().clone()
        return item

    def __len__(self):
        return len(self.labels)
'''

Sample Data Point

{'input_ids': tensor([    4,  1339,  2149, 27374,  1074,  1240,  7091, 21176,  1091,  2125,
         1051,  2240,  1096,  4334,  1017,  9391,  4341,  1030,  4837,  2623,
        18708, 15970,  1110,  5228,  1009,  2144,  2972,  2770,  1762,  1017,
         1486,  1062,  3013,  1008, 15924, 19520,  2331,  1712,  2686,  1008,
         1038,  1040,  3669,  1004,  3953,  1266, 11832, 10971,  1110,  1248,
         1042,  1294, 10789,  1009,  2156,  1065,  6591, 12435,  1287, 14178,
        30934,  1017,  2111,  2016,  7119,  1391,  3166,  4687,  1181,  4153,
        21102,  1017,  3200, 23884,  1017,  1858, 21428, 30934,  1042,  1216,
         1009,  1177,  2436,  1038, 24340,  2478, 21486, 16079,  1401,  1017,
         1355,  1058,  7650,  1038,  7472,  7855,  1942,  2943,     5,  1083,
        30969, 30966,  1171, 30981,   983,   993,   992,   993,   992,  1083,
        30969,  1083, 30969, 30960,  1171, 30981, 30966,  4332,   993,   992,
         1083, 30969,  1083, 30

<ipython-input-9-fecfbf71ebd9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased",num_labels=6).to(device)
print("Model Configurations")
print()
print(model.config)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Model Configurations

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [12]:
training_args = TrainingArguments(
    output_dir='./results',       
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-9-fecfbf71ebd9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.010500,1.157565,0.472405,0.428590,0.449258,0.473284
2,0.799000,0.880471,0.600506,0.591451,0.608309,0.595554
3,0.521600,0.947510,0.617215,0.610539,0.624572,0.612871


<ipython-input-9-fecfbf71ebd9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-9-fecfbf71ebd9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=1668, training_loss=0.8885452161304099, metrics={'train_runtime': 2375.3898, 'train_samples_per_second': 22.438, 'train_steps_per_second': 0.702, 'total_flos': 7011898333267968.0, 'train_loss': 0.8885452161304099, 'epoch': 3.0})

In [13]:
trainer.evaluate(test_dataset)
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

<ipython-input-9-fecfbf71ebd9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<ipython-input-9-fecfbf71ebd9>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           0       0.74      0.57      0.64       337
           1       0.40      0.43      0.41       357
           2       0.50      0.57      0.53       366
           3       0.71      0.45      0.55       368
           4       0.51      0.66      0.57       349
           5       0.83      0.92      0.87       417

    accuracy                           0.61      2194
   macro avg       0.62      0.60      0.60      2194
weighted avg       0.62      0.61      0.60      2194

